<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty50_Upside_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [33]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

stock_ids = stock_n100

mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE',
       'BANKBARODA', 'BATAINDIA', 'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA',
       'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB',
       'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'IBREALEST',
       'ICICIBANK', 'ICICIPRULI', 'INDIANB', 'INDIGO', 'INDUSINDBK',
       'INDUSTOWER', 'INFY', 'IRCTC', 'ITC', 'JUBLFOOD', 'KOTAKBANK',
       'LT', 'LTIM', 'M&M', 'MARUTI', 'MCDOWELL-N', 'MOTILALOFS',
       'MUTHOOTFIN', 'NAM-INDIA', 'NATIONALUM', 'NAUKRI', 'PAGEIND',
       'PFIZER', 'PIDILITIND', 'PVR', 'RELIANCE', 'SAIL', 'SBILIFE',
       'SBIN', 'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER',
       'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'VOLTAS',
       'WHIRLPOOL', 'WIPRO', 'ZEEL'], dtype=object)

In [34]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    59
1    15
Name: InPortfolio, dtype: int64

In [35]:
# consistent compounders
mypf[mypf['InPortfolio'] == 1]['CCIFlag'].value_counts()

0    11
1     4
Name: CCIFlag, dtype: int64

In [36]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [37]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [38]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 150
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='2y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_150'] = round((stock_df['Close'] - stock_df['150_SMA'])*100/stock_df['150_SMA'],2)
    stock_df['150_SMA_Lag10'] = stock_df['150_SMA'].shift(10)
    stock_df['Slope%_150'] = round((stock_df['150_SMA'] - stock_df['150_SMA_Lag10'])*100/stock_df['150_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    #stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    #stock_df['Trend'] = 'Down'
    #stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    #stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    #stock_df['EOM'] = '-ve'
    #stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Upside%52W'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','150_SMA','150_SMA_Lag10','50_SMA_Lag5'
                   #'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]   
    tmp['Stage'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Stage'].values[0]      
    df_prec_dev = df_prec_dev.append(tmp)

In [39]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Upside%52W
Date,,,,,,,,,,,
2023-02-15 00:00:00+05:30,18016.0,^NSEI,1.56,0.71,-0.46,18812.0,15294.0,77.37,22.63,54.0,4.0
2023-02-22 00:00:00+05:30,17554.0,^NSEI,-1.37,0.67,-0.39,18812.0,15294.0,64.25,35.75,37.0,7.0
2023-03-01 00:00:00+05:30,17451.0,^NSEI,-2.11,0.49,-0.56,18812.0,15294.0,61.31,38.69,38.0,8.0
2023-03-09 00:00:00+05:30,17590.0,^NSEI,-1.46,0.29,-0.26,18812.0,15294.0,65.25,34.75,46.0,7.0
2023-03-16 00:00:00+05:30,17030.0,^NSEI,-4.55,0.08,-0.59,18812.0,15294.0,49.34,50.66,34.0,10.0


In [40]:
# Upside% Calculations
df_prec_dev['Upside%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev['Upside%Avg'] = round((df_prec_dev['Upside%FC'] + df_prec_dev['Upside%52W'])/2,0)
#df_prec_dev['Upside%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Curr_PE']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%FC', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
#df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [41]:
# portfolio stocks
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Upside%FC', 'Upside%52W', 'Upside%Avg', 'EstmtFlag','DuraFlag','Stage']
df_prec_dev = df_prec_dev[cols]
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'EstmtFlag','DuraFlag','Stage']

mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Profit%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%52W'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%Avg'] = round((tmp_df['Profit%FC'] + tmp_df['Profit%52W'])/2,0)
print('qualified stocks: '+str(tmp_df.shape[0]))

cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Profit%FC', 'Profit%52W', 'Profit%Avg', 'EstmtFlag','DuraFlag','Stage']
tmp_df = tmp_df[cols]
tmp_df.sort_values(by = 'Profit%Avg', ascending=False)

qualified stocks: 15


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
54,VOLTAS.NS,876.0,0.40,-0.69,0.95,1315.0,745.0,22.89,77.11,51.0,976.0,821.0,43.4,82.90,19.0,60.0,40.0,0.0,1.0,D
11,ICICIPRULI.NS,411.0,-15.51,-2.26,-1.38,595.0,384.0,13.01,86.99,50.0,572.0,443.0,60.9,77.00,29.0,34.0,32.0,1.0,0.0,D
46,TATAPOWER.NS,202.0,-7.22,-0.92,0.00,287.0,195.0,7.88,92.12,43.0,232.0,206.0,20.4,21.10,13.0,39.0,26.0,1.0,0.0,O
2,MOTILALOFS.NS,580.0,-16.48,-1.59,-1.51,922.0,577.0,0.77,99.23,25.0,908.0,733.0,19.7,8.04,24.0,26.0,25.0,-1.0,0.0,D
38,DMART.NS,3354.0,-16.14,-1.33,-2.03,4577.0,3231.0,9.12,90.88,33.0,3983.0,3448.0,138.7,92.40,16.0,33.0,24.0,1.0,1.0,T
23,JUBLFOOD.NS,434.0,-19.81,-1.48,-1.50,639.0,427.0,3.23,96.77,40.0,554.0,484.0,87.9,68.30,14.0,32.0,23.0,1.0,1.0,B
24,RELIANCE.NS,2251.0,-9.97,-0.52,-1.17,2811.0,2237.0,2.41,97.59,35.0,2864.0,2311.0,25.3,24.00,24.0,22.0,23.0,1.0,0.0,O
9,BATAINDIA.NS,1405.0,-16.97,-1.83,-1.53,1971.0,1387.0,3.07,96.93,38.0,1998.0,1632.0,62.3,56.50,22.0,21.0,22.0,1.0,1.0,D
21,BAJFINANCE.NS,5733.0,-14.62,-1.16,-1.25,7720.0,5265.0,19.07,80.93,30.0,7318.0,6154.0,50.8,32.30,19.0,25.0,22.0,1.0,1.0,T
18,MUTHOOTFIN.NS,928.0,-11.17,-0.96,-1.39,1354.0,916.0,2.80,97.20,32.0,1227.0,1082.0,12.5,10.20,13.0,25.0,19.0,-1.0,0.0,B


In [42]:
# portfolio ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
54,VOLTAS.NS,876.0,0.40,-0.69,0.95,1315.0,745.0,22.89,77.11,51.0,976.0,821.0,43.4,82.90,19.0,60.0,40.0,0.0,1.0,D
46,TATAPOWER.NS,202.0,-7.22,-0.92,0.00,287.0,195.0,7.88,92.12,43.0,232.0,206.0,20.4,21.10,13.0,39.0,26.0,1.0,0.0,O
68,BERGEPAINT.NS,602.0,-0.50,-0.50,0.00,731.0,540.0,32.46,67.54,68.0,605.0,570.0,72.9,65.50,6.0,28.0,17.0,1.0,1.0,B
28,INFY.NS,1412.0,-6.78,-0.33,-0.52,1866.0,1351.0,11.91,88.09,28.0,1742.0,1628.0,23.0,24.70,7.0,15.0,11.0,1.0,1.0,A
24,RELIANCE.NS,2251.0,-9.97,-0.52,-1.17,2811.0,2237.0,2.41,97.59,35.0,2864.0,2311.0,25.3,24.00,24.0,22.0,23.0,1.0,0.0,O
21,BAJFINANCE.NS,5733.0,-14.62,-1.16,-1.25,7720.0,5265.0,19.07,80.93,30.0,7318.0,6154.0,50.8,32.30,19.0,25.0,22.0,1.0,1.0,T
11,ICICIPRULI.NS,411.0,-15.51,-2.26,-1.38,595.0,384.0,13.01,86.99,50.0,572.0,443.0,60.9,77.00,29.0,34.0,32.0,1.0,0.0,D
18,MUTHOOTFIN.NS,928.0,-11.17,-0.96,-1.39,1354.0,916.0,2.80,97.20,32.0,1227.0,1082.0,12.5,10.20,13.0,25.0,19.0,-1.0,0.0,B
12,NAUKRI.NS,3397.0,-12.85,-1.26,-1.44,4865.0,3363.0,2.27,97.73,36.0,4672.0,4154.0,78.8,60.40,12.0,17.0,14.0,1.0,0.0,B
23,JUBLFOOD.NS,434.0,-19.81,-1.48,-1.50,639.0,427.0,3.23,96.77,40.0,554.0,484.0,87.9,68.30,14.0,32.0,23.0,1.0,1.0,B


In [43]:
# wait for max profit

tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
38,DMART.NS,3354.0,-16.14,-1.33,-2.03,4577.0,3231.0,9.12,90.88,33.0,3983.0,3448.0,138.7,92.4,16.0,33.0,24.0,1.0,1.0,T
23,JUBLFOOD.NS,434.0,-19.81,-1.48,-1.50,639.0,427.0,3.23,96.77,40.0,554.0,484.0,87.9,68.3,14.0,32.0,23.0,1.0,1.0,B
9,BATAINDIA.NS,1405.0,-16.97,-1.83,-1.53,1971.0,1387.0,3.07,96.93,38.0,1998.0,1632.0,62.3,56.5,22.0,21.0,22.0,1.0,1.0,D
21,BAJFINANCE.NS,5733.0,-14.62,-1.16,-1.25,7720.0,5265.0,19.07,80.93,30.0,7318.0,6154.0,50.8,32.3,19.0,25.0,22.0,1.0,1.0,T
8,HDFCLIFE.NS,466.0,-15.05,-0.73,-1.87,612.0,466.0,0.24,99.76,28.0,678.0,544.0,89.3,66.2,25.0,12.0,18.0,1.0,1.0,A
68,BERGEPAINT.NS,602.0,-0.50,-0.50,0.00,731.0,540.0,32.46,67.54,68.0,605.0,570.0,72.9,65.5,6.0,28.0,17.0,1.0,1.0,B
28,INFY.NS,1412.0,-6.78,-0.33,-0.52,1866.0,1351.0,11.91,88.09,28.0,1742.0,1628.0,23.0,24.7,7.0,15.0,11.0,1.0,1.0,A


In [44]:
# keep a watch
tmp_df[((tmp_df['EstmtFlag'] == 0) & (tmp_df['DuraFlag'] != -1)) | ((tmp_df['DuraFlag'] == 0) & (tmp_df['EstmtFlag'] != -1))].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
54,VOLTAS.NS,876.0,0.40,-0.69,0.95,1315.0,745.0,22.89,77.11,51.0,976.0,821.0,43.4,82.9,19.0,60.0,40.0,0.0,1.0,D
11,ICICIPRULI.NS,411.0,-15.51,-2.26,-1.38,595.0,384.0,13.01,86.99,50.0,572.0,443.0,60.9,77.0,29.0,34.0,32.0,1.0,0.0,D
46,TATAPOWER.NS,202.0,-7.22,-0.92,0.00,287.0,195.0,7.88,92.12,43.0,232.0,206.0,20.4,21.1,13.0,39.0,26.0,1.0,0.0,O
24,RELIANCE.NS,2251.0,-9.97,-0.52,-1.17,2811.0,2237.0,2.41,97.59,35.0,2864.0,2311.0,25.3,24.0,24.0,22.0,23.0,1.0,0.0,O
12,NAUKRI.NS,3397.0,-12.85,-1.26,-1.44,4865.0,3363.0,2.27,97.73,36.0,4672.0,4154.0,78.8,60.4,12.0,17.0,14.0,1.0,0.0,B


In [45]:
# sell as soon as you can
tmp_df[(tmp_df['EstmtFlag'] == -1) | (tmp_df['DuraFlag'] == -1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
2,MOTILALOFS.NS,580.0,-16.48,-1.59,-1.51,922.0,577.0,0.77,99.23,25.0,908.0,733.0,19.7,8.04,24.0,26.0,25.0,-1.0,0.0,D
18,MUTHOOTFIN.NS,928.0,-11.17,-0.96,-1.39,1354.0,916.0,2.80,97.20,32.0,1227.0,1082.0,12.5,10.20,13.0,25.0,19.0,-1.0,0.0,B
34,DIVISLAB.NS,2827.0,-16.19,-1.96,-2.03,4530.0,2751.0,4.27,95.73,43.0,3393.0,3671.0,45.2,31.30,-8.0,23.0,8.0,-1.0,1.0,B


In [46]:
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Med_PE', 'Curr_PE','Upside%FC', 'Upside%52W', 'Upside%Avg','EstmtFlag','DuraFlag','Stage']
df_prec_dev = df_prec_dev[cols]

In [47]:
# prospect consistent compounders (CC)
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 3


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
23,JUBLFOOD.NS,434.0,-19.81,-1.48,-1.50,639.0,427.0,3.23,96.77,40.0,554.0,87.9,68.3,28.0,47.0,38.0,1.0,1.0,B
40,PAGEIND.NS,37427.0,-17.39,-1.62,-1.53,53631.0,36267.0,6.68,93.32,43.0,43791.0,80.8,1000.0,17.0,43.0,30.0,1.0,1.0,O
28,INFY.NS,1412.0,-6.78,-0.33,-0.52,1866.0,1351.0,11.91,88.09,28.0,1742.0,23.0,24.7,23.0,32.0,28.0,1.0,1.0,A


In [48]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 12


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
19,BAJAJFINSV.NS,1290.0,-17.45,-0.64,-1.76,1805.0,1093.0,27.71,72.29,35.0,1691.0,39.2,34.2,31.0,40.0,36.0,1.0,0.0,D
13,INDUSINDBK.NS,1023.0,-11.51,0.43,-1.40,1264.0,771.0,51.11,48.89,33.0,1402.0,22.8,11.7,37.0,24.0,30.0,1.0,1.0,O
40,PAGEIND.NS,37427.0,-17.39,-1.62,-1.53,53631.0,36267.0,6.68,93.32,43.0,43791.0,80.8,1000.0,17.0,43.0,30.0,1.0,1.0,O
33,WHIRLPOOL.NS,1343.0,-12.32,-1.96,-1.18,1839.0,1254.0,15.25,84.75,53.0,1629.0,72.2,70.9,21.0,37.0,29.0,1.0,0.0,O
14,SBIN.NS,524.0,-7.32,0.00,-1.44,626.0,435.0,46.41,53.59,41.0,716.0,13.2,9.9,37.0,19.0,28.0,1.0,0.0,O
29,CIPLA.NS,889.0,-16.32,-0.66,-2.00,1173.0,872.0,5.53,94.47,29.0,1092.0,NaN,26.9,23.0,32.0,28.0,1.0,1.0,D
20,EICHERMOT.NS,3071.0,-8.89,0.06,-0.56,3850.0,2321.0,49.07,50.93,42.0,3916.0,40.1,32.1,28.0,25.0,26.0,1.0,1.0,T
17,ICICIBANK.NS,825.0,-6.67,0.23,-0.82,953.0,673.0,54.29,45.71,36.0,1094.0,24.2,18.1,33.0,16.0,24.0,1.0,1.0,O
26,TATACONSUM.NS,696.0,-9.66,-0.78,-0.96,849.0,688.0,4.75,95.25,36.0,871.0,NaN,61.3,25.0,22.0,24.0,1.0,1.0,O
25,M&M.NS,1164.0,-9.31,0.00,-0.61,1388.0,757.0,64.58,35.42,27.0,1475.0,20.5,16.3,27.0,19.0,23.0,1.0,1.0,O


In [49]:
# heavy upside ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
20,EICHERMOT.NS,3071.0,-8.89,0.06,-0.56,3850.0,2321.0,49.07,50.93,42.0,3916.0,40.1,32.1,28.0,25.0,26.0,1.0,1.0,T
25,M&M.NS,1164.0,-9.31,0.00,-0.61,1388.0,757.0,64.58,35.42,27.0,1475.0,20.5,16.3,27.0,19.0,23.0,1.0,1.0,O
17,ICICIBANK.NS,825.0,-6.67,0.23,-0.82,953.0,673.0,54.29,45.71,36.0,1094.0,24.2,18.1,33.0,16.0,24.0,1.0,1.0,O
26,TATACONSUM.NS,696.0,-9.66,-0.78,-0.96,849.0,688.0,4.75,95.25,36.0,871.0,NaN,61.3,25.0,22.0,24.0,1.0,1.0,O
35,AUBANK.NS,581.0,-7.81,0.00,-0.97,717.0,550.0,18.44,81.56,34.0,698.0,NaN,114.0,20.0,23.0,22.0,1.0,0.0,D
22,AXISBANK.NS,830.0,-2.58,0.94,-1.13,962.0,625.0,60.83,39.17,39.0,1060.0,31.3,12.4,28.0,16.0,22.0,1.0,0.0,A
33,WHIRLPOOL.NS,1343.0,-12.32,-1.96,-1.18,1839.0,1254.0,15.25,84.75,53.0,1629.0,72.2,70.9,21.0,37.0,29.0,1.0,0.0,O
13,INDUSINDBK.NS,1023.0,-11.51,0.43,-1.40,1264.0,771.0,51.11,48.89,33.0,1402.0,22.8,11.7,37.0,24.0,30.0,1.0,1.0,O
14,SBIN.NS,524.0,-7.32,0.00,-1.44,626.0,435.0,46.41,53.59,41.0,716.0,13.2,9.9,37.0,19.0,28.0,1.0,0.0,O
40,PAGEIND.NS,37427.0,-17.39,-1.62,-1.53,53631.0,36267.0,6.68,93.32,43.0,43791.0,80.8,1000.0,17.0,43.0,30.0,1.0,1.0,O


In [50]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 79]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Premium%', ascending=True)

qualified stocks: 6


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
26,TATACONSUM.NS,696.0,-9.66,-0.78,-0.96,849.0,688.0,4.75,95.25,36.0,871.0,NaN,61.3,25.0,22.0,24.0,1.0,1.0,O
29,CIPLA.NS,889.0,-16.32,-0.66,-2.00,1173.0,872.0,5.53,94.47,29.0,1092.0,NaN,26.9,23.0,32.0,28.0,1.0,1.0,D
40,PAGEIND.NS,37427.0,-17.39,-1.62,-1.53,53631.0,36267.0,6.68,93.32,43.0,43791.0,80.8,1000.0,17.0,43.0,30.0,1.0,1.0,O
70,IRCTC.NS,603.0,-11.22,-0.15,-0.48,826.0,566.0,14.17,85.83,38.0,571.0,NaN,1000.0,-5.0,37.0,16.0,1.0,1.0,O
33,WHIRLPOOL.NS,1343.0,-12.32,-1.96,-1.18,1839.0,1254.0,15.25,84.75,53.0,1629.0,72.2,70.9,21.0,37.0,29.0,1.0,0.0,O
35,AUBANK.NS,581.0,-7.81,0.00,-0.97,717.0,550.0,18.44,81.56,34.0,698.0,NaN,114.0,20.0,23.0,22.0,1.0,0.0,D


In [51]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 2


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
73,ITC.NS,380.0,11.76,1.76,1.66,392.0,233.0,92.45,7.55,54.0,350.0,NaN,25.9,-8.0,3.0,-2.0,1.0,0.0,O
71,LT.NS,2155.0,5.48,1.17,0.28,2273.0,1453.0,85.60,14.40,49.0,2036.0,NaN,30.1,-6.0,5.0,-0.0,1.0,0.0,O


In [52]:
# 52W high ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
73,ITC.NS,380.0,11.76,1.76,1.66,392.0,233.0,92.45,7.55,54.0,350.0,NaN,25.9,-8.0,3.0,-2.0,1.0,0.0,O
71,LT.NS,2155.0,5.48,1.17,0.28,2273.0,1453.0,85.60,14.40,49.0,2036.0,NaN,30.1,-6.0,5.0,-0.0,1.0,0.0,O


In [53]:
# prospects turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > -.5) & (tmp_df['Slope%_150'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 4


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
55,MARUTI.NS,8526.0,-2.83,-0.19,0.14,9528.0,7055.0,59.48,40.52,44.0,9356.0,NaN,34.7,10.0,12.0,11.0,1.0,1.0,O
45,HINDUNILVR.NS,2464.0,-4.35,-0.35,-0.47,2722.0,1906.0,68.38,31.62,43.0,2832.0,NaN,58.5,15.0,10.0,12.0,1.0,1.0,A
70,IRCTC.NS,603.0,-11.22,-0.15,-0.48,826.0,566.0,14.17,85.83,38.0,571.0,NaN,1000.0,-5.0,37.0,16.0,1.0,1.0,O
64,PIDILITIND.NS,2345.0,-9.11,-0.43,-1.06,2902.0,1991.0,38.86,61.14,54.0,2448.0,70.6,95.9,4.0,24.0,14.0,1.0,1.0,O


In [54]:
# prospects turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%52W', ascending=False)

qualified stocks: 1


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
55,MARUTI.NS,8526.0,-2.83,-0.19,0.14,9528.0,7055.0,59.48,40.52,44.0,9356.0,NaN,34.7,10.0,12.0,11.0,1.0,1.0,O


In [55]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 1


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
66,LTIM.NS,4561.0,-0.54,0.02,0.62,6234.0,3761.0,32.36,67.64,42.0,4663.0,26.6,39.0,2.0,37.0,20.0,1.0,0.0,O


In [56]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 4


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
55,MARUTI.NS,8526.0,-2.83,-0.19,0.14,9528.0,7055.0,59.48,40.52,44.0,9356.0,NaN,34.7,10.0,12.0,11.0,1.0,1.0,O
57,ASIANPAINT.NS,2859.0,-7.81,-1.10,-0.95,3566.0,2576.0,28.58,71.42,55.0,3131.0,73.4,72.0,10.0,25.0,18.0,1.0,1.0,T
33,WHIRLPOOL.NS,1343.0,-12.32,-1.96,-1.18,1839.0,1254.0,15.25,84.75,53.0,1629.0,72.2,70.9,21.0,37.0,29.0,1.0,0.0,O
40,PAGEIND.NS,37427.0,-17.39,-1.62,-1.53,53631.0,36267.0,6.68,93.32,43.0,43791.0,80.8,1000.0,17.0,43.0,30.0,1.0,1.0,O


In [57]:
# star stocks performance

star_stocks = ['BAJFINANCE.NS', 'EICHERMOT.NS', 'FEDERALBNK.NS', 'CANBK.NS',
       'AUBANK.NS', 'TCS.NS', 'INDUSINDBK.NS', 'TECHM.NS', 'ICICIBANK.NS',
       'AXISBANK.NS', 'HCLTECH.NS', 'WIPRO.NS', 'TATAPOWER.NS', 'HDFC.NS',
       'HDFCBANK.NS', 'HINDUNILVR.NS', 'EMAMILTD.NS', 'INDIANB.NS',
       'BANKBARODA.NS', 'INFY.NS', 'KOTAKBANK.NS', 'SBIN.NS',
       'TATACONSUM.NS', 'ITC.NS', 'LT.NS']

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
21,BAJFINANCE.NS,5733.0,-14.62,-1.16,-1.25,7720.0,5265.0,19.07,80.93,30.0,7318.0,50.8,32.30,28.0,35.0,32.0,1.0,1.0,T
13,INDUSINDBK.NS,1023.0,-11.51,0.43,-1.40,1264.0,771.0,51.11,48.89,33.0,1402.0,22.8,11.70,37.0,24.0,30.0,1.0,1.0,O
46,TATAPOWER.NS,202.0,-7.22,-0.92,0.00,287.0,195.0,7.88,92.12,43.0,232.0,20.4,21.10,15.0,42.0,28.0,1.0,0.0,O
28,INFY.NS,1412.0,-6.78,-0.33,-0.52,1866.0,1351.0,11.91,88.09,28.0,1742.0,23.0,24.70,23.0,32.0,28.0,1.0,1.0,A
14,SBIN.NS,524.0,-7.32,0.00,-1.44,626.0,435.0,46.41,53.59,41.0,716.0,13.2,9.90,37.0,19.0,28.0,1.0,0.0,O
20,EICHERMOT.NS,3071.0,-8.89,0.06,-0.56,3850.0,2321.0,49.07,50.93,42.0,3916.0,40.1,32.10,28.0,25.0,26.0,1.0,1.0,T
17,ICICIBANK.NS,825.0,-6.67,0.23,-0.82,953.0,673.0,54.29,45.71,36.0,1094.0,24.2,18.10,33.0,16.0,24.0,1.0,1.0,O
26,TATACONSUM.NS,696.0,-9.66,-0.78,-0.96,849.0,688.0,4.75,95.25,36.0,871.0,NaN,61.30,25.0,22.0,24.0,1.0,1.0,O
22,AXISBANK.NS,830.0,-2.58,0.94,-1.13,962.0,625.0,60.83,39.17,39.0,1060.0,31.3,12.40,28.0,16.0,22.0,1.0,0.0,A
35,AUBANK.NS,581.0,-7.81,0.00,-0.97,717.0,550.0,18.44,81.56,34.0,698.0,NaN,114.00,20.0,23.0,22.0,1.0,0.0,D


In [58]:
# star stocks ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
73,ITC.NS,380.0,11.76,1.76,1.66,392.0,233.0,92.45,7.55,54.0,350.0,NaN,25.90,-8.0,3.0,-2.0,1.0,0.0,O
65,HCLTECH.NS,1086.0,5.37,1.16,0.54,1150.0,857.0,78.29,21.71,43.0,1117.0,16.3,20.40,3.0,6.0,4.0,1.0,1.0,A
71,LT.NS,2155.0,5.48,1.17,0.28,2273.0,1453.0,85.60,14.40,49.0,2036.0,NaN,30.10,-6.0,5.0,-0.0,1.0,0.0,O
61,TCS.NS,3215.0,-0.45,0.19,0.12,3695.0,2915.0,38.51,61.49,35.0,3385.0,28.1,28.90,5.0,15.0,10.0,1.0,0.0,O
46,TATAPOWER.NS,202.0,-7.22,-0.92,0.00,287.0,195.0,7.88,92.12,43.0,232.0,20.4,21.10,15.0,42.0,28.0,1.0,0.0,O
60,INDIANB.NS,266.0,6.10,2.79,-0.35,305.0,132.0,77.63,22.37,43.0,284.0,7.4,6.56,7.0,15.0,11.0,1.0,1.0,O
44,HDFC.NS,2535.0,-0.64,0.67,-0.38,2775.0,2053.0,66.72,33.28,35.0,2940.0,23.0,18.60,16.0,9.0,12.0,1.0,0.0,A
39,HDFCBANK.NS,1554.0,-0.26,0.64,-0.43,1696.0,1281.0,65.78,34.22,36.0,1813.0,26.0,19.80,17.0,9.0,13.0,1.0,0.0,A
45,HINDUNILVR.NS,2464.0,-4.35,-0.35,-0.47,2722.0,1906.0,68.38,31.62,43.0,2832.0,NaN,58.50,15.0,10.0,12.0,1.0,1.0,A
28,INFY.NS,1412.0,-6.78,-0.33,-0.52,1866.0,1351.0,11.91,88.09,28.0,1742.0,23.0,24.70,23.0,32.0,28.0,1.0,1.0,A


In [59]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != 1) | (tmp_df['DuraFlag'] == -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
0,IBREALEST.NS,56.0,-26.32,0.00,-3.03,118.0,53.0,4.62,95.38,38.0,95.0,11.3,1000.00,70.0,111.0,90.0,-1.0,-1.0,B
1,ZEEL.NS,193.0,-21.02,-1.23,-2.33,299.0,183.0,8.36,91.64,40.0,303.0,24.6,32.60,57.0,55.0,56.0,-1.0,1.0,O
7,NAM-INDIA.NS,219.0,-15.65,-1.54,-1.29,340.0,215.0,3.44,96.56,43.0,322.0,29.9,19.50,47.0,55.0,51.0,-1.0,1.0,O
16,HINDALCO.NS,387.0,-11.01,0.00,-1.57,626.0,314.0,23.43,76.57,30.0,514.0,10.4,7.43,33.0,62.0,48.0,-1.0,1.0,A
4,EMAMILTD.NS,363.0,-17.86,-1.13,-1.23,512.0,363.0,0.03,99.97,28.0,550.0,49.6,18.90,52.0,41.0,46.0,-1.0,1.0,T
10,INDUSTOWER.NS,152.0,-18.12,-2.15,-2.41,229.0,137.0,16.63,83.37,37.0,213.0,NaN,15.00,40.0,51.0,46.0,-1.0,1.0,O
3,GODREJIND.NS,413.0,-6.42,-0.45,-0.47,502.0,405.0,7.94,92.06,41.0,638.0,39.5,15.20,54.0,22.0,38.0,0.0,1.0,B
6,PFIZER.NS,3592.0,-14.67,-0.67,-1.90,4599.0,3592.0,-0.02,100.02,18.0,5297.0,36.5,107.00,47.0,28.0,38.0,0.0,1.0,O
15,PVR.NS,1571.0,-10.36,-2.11,-1.16,2193.0,1497.0,10.57,89.43,46.0,2106.0,51.2,1000.00,34.0,40.0,37.0,1.0,-1.0,O
50,WIPRO.NS,376.0,-5.69,-0.50,-0.25,605.0,374.0,1.00,99.00,30.0,425.0,19.0,18.20,13.0,61.0,37.0,-1.0,0.0,O


In [60]:
# cross-check ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
67,TECHM.NS,1115.0,6.38,0.48,0.85,1520.0,935.0,30.74,69.26,57.0,1113.0,18.7,20.80,-0.0,36.0,18.0,-1.0,1.0,B
51,HAVELLS.NS,1184.0,-3.94,-0.32,0.75,1388.0,1067.0,36.59,63.41,45.0,1328.0,63.7,69.60,12.0,17.0,14.0,-1.0,1.0,T
27,TATAMOTORS.NS,414.0,-2.79,-0.47,0.71,490.0,372.0,35.68,64.32,41.0,517.0,19.2,1000.00,25.0,18.0,22.0,1.0,-1.0,B
63,GODREJCP.NS,924.0,3.63,0.45,0.33,948.0,685.0,91.01,8.99,52.0,958.0,NaN,57.20,4.0,3.0,4.0,-1.0,0.0,A
62,ULTRACEMCO.NS,7127.0,4.14,0.60,0.17,7427.0,5148.0,86.85,13.15,47.0,7420.0,33.7,34.20,4.0,4.0,4.0,-1.0,1.0,O
69,SAIL.NS,86.0,3.55,0.00,0.00,108.0,63.0,51.00,49.00,49.0,85.0,6.9,10.80,-1.0,26.0,12.0,-1.0,1.0,O
53,NATIONALUM.NS,81.0,5.71,1.30,0.00,126.0,66.0,25.67,74.33,50.0,90.0,10.3,7.60,11.0,56.0,34.0,0.0,1.0,O
50,WIPRO.NS,376.0,-5.69,-0.50,-0.25,605.0,374.0,1.00,99.00,30.0,425.0,19.0,18.20,13.0,61.0,37.0,-1.0,0.0,O
59,COLPAL.NS,1500.0,-3.16,-0.26,-0.27,1660.0,1443.0,26.29,73.71,58.0,1606.0,NaN,94.60,7.0,11.0,9.0,-1.0,1.0,D
3,GODREJIND.NS,413.0,-6.42,-0.45,-0.47,502.0,405.0,7.94,92.06,41.0,638.0,39.5,15.20,54.0,22.0,38.0,0.0,1.0,B
